Dans un premier temps, on fait une analyse quantitative pour déterminer à quel niveau les deux retrievers sont complémentaries. On essaie ensuite de déterminer des seuils pour le score qui nous permetteraient de choisir un des deux/en faire un mélange pour incrementer la performance.

In [23]:
import json
import numpy as np
import pandas as pd

In [24]:
dense_json = json.load(open("f615_detailed_results.json"))
sparse_json = json.load(open("9abf_detailed_results.json"))

In [25]:
dense_errors = dense_json["errors"]
sparse_errors = sparse_json["errors"]
dense_successes = dense_json["successes"]
sparse_successes = sparse_json["successes"]

In [26]:
sparse_bon = set(sparse_successes.keys())
sparse_mauvais = set(sparse_errors.keys())
dense_bon = set(dense_successes.keys())
dense_mauvais = set(dense_errors.keys())

### Erreurs communes au deux retrievers

Etant donné notre échantillon de réponses correctes et d'erreurs pour le retriever sparse et le retriever dense, vérifions tout d'abord si les deux se trompent souvent sur les memes questions et si oui, en quel pourcentage par rapport aux errors commis au total.

In [27]:
common_errors = list(dense_mauvais.intersection(sparse_mauvais))

In [28]:
print(len(common_errors))
print(len(common_errors)/(len(dense_errors)+len(sparse_errors)))

77
0.2664359861591695


*Dans cet échantillon, 26% des erreurs commis sont commis par les deux retrivers. Il est peut etre intéressant d'aller plus loin dans l'analyse de ces erreurs qui semblent très difficiles à résoudre pour les deux retrievers.*

### Evaluation des cas où le retriever dense fait des erreurs et le sparse trouve la bonne réponse

In [29]:
dense_fails_sparse_good = list(sparse_bon.intersection(dense_mauvais))
print(len(dense_fails_sparse_good))

95


*Pour 95 questions, le retriever sparse donne des bonnes réponses là où le dense se trompe.*

In [30]:
print(len(dense_fails_sparse_good)/len(dense_mauvais))

0.5523255813953488


**Sur les 172 erreurs faites par le retriever dense, dans 55% des cas le sparse peut donner la bonne réponse, et il pourrait donc etre utile de mélanger les deux.**

### Evaluation des cas où le retriever sparse fait des erreurs et le dense trouve la bonne réponse

In [31]:
sparse_fails_dense_good = list(sparse_mauvais.intersection(dense_bon))
print(len(sparse_fails_dense_good))

40


*Pour 40 questions, le retriever dense donne des bonnes réponses là où le sparse se trompe.*

In [32]:
print(len(sparse_fails_dense_good)/len(sparse_mauvais))

0.3418803418803419


**Sur les 117 erreurs faites par le retriever sparse, dans 34% des cas le dense peut donner la bonne réponse.**

### Conclusion:

Le retriever sparse, si mélangé au retriever dense, peut aider à l'améliorer (le sparse donne globalement des meilleures réponses donc c'est assez normal). Dans 34% des cas, le retriever dense peut apporter des bonnes réponses là où le sparse se trompe. Globalement, on ne peut pas parler de complémentarité exacte (dans 26% des cas les deux se trompent et un mélange des deux n'apporteraient donc aucune amélioration), mais coupler le sparse au dense peut augmenter le nombre de bonnes réponses par rapport au dense seul.

### Suite de l'analyse: peut-on trouver un seuil pour 'score' ou 'proba' qui nous indique qu'il faudrait utiliser le retriever sparse pour améliorer le dense?

*exemple de ce qu'on souhaiterait avoir : if [score (dense) < 0.2 et score (sparse) > 0.7 ]: use sparse*

Voici un dataframe qui contient la liste des questions où le retriever sparse améliore le dense, avec les scores et les probas correspondantes:

In [33]:
#premiere colonne du csv: liste des questions fausses pour dense, bonnes pour sparse
col1 = [dense_fails_sparse_good[k] for k in range(len(dense_fails_sparse_good))] 

#deuxième colonne csv: liste des scores correspondants pour dense
from statistics import mean
tmp = [dense_errors[dense_fails_sparse_good[k]]['pred_fiches'] for k in range(len(col1))]
L=[]
M =[]
for j in range(len(tmp)):
        l = [tmp[j][k][2] for k in range(len(tmp[j]))]
        m = mean(l)
        L.append(m)
#troisième colonne: liste des scores correspondants pour sparse
tmp = [sparse_successes[dense_fails_sparse_good[k]]['pred_fiches'] for k in range(len(col1))]
for j in range(len(tmp)):
    l = [tmp[j][k][2] for k in range(len(tmp[j]))]
    m1 = mean(l)
    M.append(m1)

#colonnes des probas

tmp = [dense_errors[dense_fails_sparse_good[k]]['pred_fiches'] for k in range(len(col1))]
A=[]
B =[]
for j in range(len(tmp)):
        l = [tmp[j][k][3] for k in range(len(tmp[j]))]
        a = mean(l)
        A.append(a)
#troisième colonne: liste des scores correspondants pour sparse
tmp = [sparse_successes[dense_fails_sparse_good[k]]['pred_fiches'] for k in range(len(col1))]
for j in range(len(tmp)):
    l = [tmp[j][k][3] for k in range(len(tmp[j]))]
    b = mean(l)
    B.append(b)
    

df = pd.DataFrame(list(zip(col1, L, M,A,B)), columns =['Questions', 'score moyen_dense','score moyen_sparse','proba moyenne_dense','proba moyen_sparse']) 
df.head()

,Questions,score moyen_dense,score moyen_sparse,proba moyenne_dense,proba moyen_sparse
0,quellel est la date limite pour demander une b...,0.480306,19.480194,0.740153,0.918204
1,Sachant que le conjoint est exonéré des frais ...,0.655484,39.257036,0.827742,0.989863
2,"bonjour, Mon époux vient de rectifier son nom ...",0.575318,41.258762,0.787659,0.994167
3,bonjour puis je vous envoyer le papier par mai...,0.449137,24.248285,0.724568,0.951416
4,"Bonjour, Etant en pleine démission de mon CDI,...",0.636567,26.511257,0.818283,0.963961


In [34]:
#df.to_csv('Erreurs du retriever dense et succes du retriever sparse.csv',index=False)

To do:
- remplacer la colonne 'score moyen_sparse' par le score exact qui correspond à la bonne fiche parmi les fiches prédites
- ramener le score du sparse sur (0,1)
- **au vu de l'analyse faite sur le score de BM25 (i.e. les scores des deux retrievers ont un sens assez différent), est-il vraiment pertinant de faire un mélange des deux / établir un lien entre les deux? Serait-il peut etre plus intéressant de considérer les deux indépendament et établir un seuil pour le score à partir du quel on affichera la réponse ou pas?**